### Advent of Code 2018

Starting on 1 December 2018 and, every day, until the 25 December 2018, a new two-part puzzle is available. Providing a correct answer to the first part of the puzzle will give you a gold star. If you complete both parts, you receive... two gold stars. The description for the second part of the puzzle is only shown after solving part one. 

I will not copy here the complete descriptions of the puzzle. Follow the links in each section to understand what is asked in each puzzle.

Let's start with some imports that are needed for the solutions 

In [79]:
import os
from collections import defaultdict
from itertools import cycle

#### [Day 1](https://adventofcode.com/2018/day/1): Chronal Calibration

The input for the first puzzle is a list of integers, one per line, representing the frequency changes. Let's have a look at the first few lines in the input file.

In [68]:
with open('input/p1.txt', 'r') as f:
   chgs = f.read().strip().split()
        
chgs[:10]

['-8', '-18', '-14', '-10', '-7', '-10', '-16', '+19', '-5', '-7']

##### Part 1
> Starting with a frequency of zero, what is the resulting frequency after all of the changes in frequency have been applied?

That's easy! The resulting frequency corresponds to the sum of all the frequency changes

In [66]:
def solve():
    with open('input/p1.txt', 'r') as f:
        chgs = [int(x) for x in open("input/p1.txt").readlines()]
        
    return sum(chgs)

solve()

497

Submiting the output above will earn us our first star. On to part 2

##### Part 2
> What is the first frequency your device reaches twice?

Reading carefully the description we realize that we have to loop multiple times over the input frequency changes. After a quick search I found __[itertools.cycle](https://docs.python.org/3.6/library/itertools.html#itertools.cycle)__. This allow us to make an iterator that repeats indefinitely over the elementos of the input list. In each iteration, the resulting frequency is added to the set if not visited previously. The process stops when a match is found in the set.  

<b>Important:</b> Remember to add the initial frequency (zero) to the set. Otherwise the solution fails for the trivial case: [-1, +1]

In [76]:
def solve2():
    with open('input/p1.txt', 'r') as f:
        chgs = [int(x) for x in open("input/p1.txt").readlines()]
    
    f = 0                  #starting frequency
    visited = set([f])     #store the starting frequency
    for c in cycle(chgs): 
        f += c
        if f in visited:
            return f
        else:
            visited.add(f)

solve2()

558

That's our second star. The first puzzle is complete.

#### [Day 2](https://adventofcode.com/2018/day/2): Inventory Management System

